In [2]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

In [3]:
#load data
inc_init = pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/avg_individual_income.csv")
fac_init = pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/child_care_regulated.csv")
emp_init = pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/employment_rate.csv")
pop_init = pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/population.csv")
loc= pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/potential_locations.csv")

fac_init.rename(columns={'zip_code': 'zipcode'}, inplace=True)
inc_init.rename(columns={'ZIP code': 'zipcode'}, inplace=True)

#clean zipcode
def clean_zipcode_column(df, column_name):
    """
    """
    #convert zipcode to string and take the first 5 characters
    df.loc[:, column_name] = df[column_name].astype(str).str.slice(0, 5)
    return df

fac_init = clean_zipcode_column(fac_init, 'zipcode')
pop_init = clean_zipcode_column(pop_init, 'zipcode')
emp_init = clean_zipcode_column(emp_init, 'zipcode')
inc_init = clean_zipcode_column(inc_init, 'zipcode')

Creating new columns: Capacity for age 0-5 and capacity for age 5-12 by summing existing age group data. Fill with 0 capacity if data has N/A entry

In [4]:
fac_init.rename(columns={'zip_code': 'zipcode'}, inplace=True)
fac_init['cap_0_5'] = (
    fac_init['infant_capacity'].fillna(0) +
    fac_init['toddler_capacity'].fillna(0) +
    fac_init['preschool_capacity'].fillna(0)
)
fac_init['cap_5_12'] = fac_init['school_age_capacity'].fillna(0)
fac_init['total_cap'] = fac_init['cap_0_5'] + fac_init['cap_5_12']


Creating new columns: Population by age group 0-5 and 5-12

In [6]:
# --- Define child population columns manually ---
# Replace NaNs with 0 in case some ZIPs are missing age data
pop_init = pop_init.fillna(0)

# 0–5 = '-5' (since that's your label)
pop_init['pop_0_5'] = pop_init['-5']

# 5–12 = '5-9' + part of '10-14' (approximation)
# We'll assume 3/5 of the 10–14 group are aged 10–12 (since 3 out of 5 years)
pop_init['pop_5_12'] = pop_init['5-9'] + 0.6 * pop_init['10-14']

# total children 0–12
pop_init['pop_children'] = pop_init['pop_0_5'] + pop_init['pop_5_12']

Align datsets
- since facility location data are on a facility level, we groupby zipcode to align with other zipcode level data

In [7]:
#zipcode level
fac_zip = (
    fac_init
    .groupby('zipcode')[['cap_0_5', 'cap_5_12', 'total_cap']]
    .sum()
    .reset_index()
)

#reindex and fill with mean
all_zips = sorted(set(emp_init['zipcode']) | set(inc_init['zipcode']) | set(pop_init['zipcode']) | set(fac_zip['zipcode']))

def reindex_and_fill(df, zip_col, value_cols):
    df = df.set_index(zip_col).reindex(all_zips)
    for c in value_cols:
        df[c] = df[c].fillna(df[c].mean())  # fill missing with column mean
    df = df.reset_index().rename(columns={'index': 'zipcode'})
    return df
emp = reindex_and_fill(emp_init, 'zipcode', ['employment rate'])
inc = reindex_and_fill(inc_init, 'zipcode', ['average income'])
pop = reindex_and_fill(pop_init, 'zipcode', ['pop_0_5', 'pop_5_12', 'pop_children'])
fac = fac_zip.set_index('zipcode').reindex(all_zips).fillna(0).reset_index()

for df in [pop, fac, loc, emp, inc]:
    df['zipcode'] = df['zipcode'].astype(str).str.zfill(5)



Check missing value and shape

In [8]:

merged = (
    pop.merge(emp, on='zipcode', how='left')
            .merge(inc, on='zipcode', how='left')
            .merge(fac, on='zipcode', how='left')
)

merged[['cap_0_5', 'cap_5_12', 'total_cap']] = merged[['cap_0_5', 'cap_5_12', 'total_cap']].fillna(0)



merged_clean = merged[[
    'zipcode',
    'pop_0_5',
    'pop_5_12',
    'pop_children',
    'employment rate',
    'average income',
    'total_cap',
    'cap_0_5',
    'cap_5_12'
]]

print("Remaining missing values:\n", merged_clean.isna().sum())

# --- Quick sanity check ---
print("\nExample rows:")
display(merged_clean.head(10))

Remaining missing values:
 zipcode            0
pop_0_5            0
pop_5_12           0
pop_children       0
employment rate    0
average income     0
total_cap          0
cap_0_5            0
cap_5_12           0
dtype: int64

Example rows:


,zipcode,pop_0_5,pop_5_12,pop_children,employment rate,average income,total_cap,cap_0_5,cap_5_12
0,10001,744.0,1349.2,2093.2,0.595097,102878.033603,585.0,0.0,585.0
1,10002,2142.0,4964.8,7106.8,0.520662,59604.041165,4526.0,18.0,4508.0
2,10003,1440.0,1605.8,3045.8,0.497244,114273.049645,1995.0,0.0,1995.0
3,10004,433.0,278.6,711.6,0.506661,132004.310345,263.0,0.0,263.0
4,10005,484.0,341.4,825.4,0.665833,121437.713311,39.0,0.0,39.0
5,10006,128.0,141.0,269.0,0.631692,126377.118644,156.0,14.0,142.0
6,10007,605.0,625.0,1230.0,0.528910,138853.904282,284.0,0.0,284.0
7,10009,1896.0,2759.0,4655.0,0.514567,77133.233533,1678.0,18.0,1660.0
8,10010,1422.0,2160.8,3582.8,0.492749,116272.698810,234.0,0.0,234.0
9,10011,1209.0,2018.4,3227.4,0.557000,120420.792079,1950.0,42.0,1908.0


## Objective and Scope

This optimization model identifies the minimum total budget required to expand and build childcare facilities across ZIP codes in order to satisfy policy coverage thresholds for children under age five (0–5) and ages 5–12.
Each ZIP code is optimized independently, treating it as a local planning unit.
The model selects:

- how much to expand existing facilities, and

- where and what size of new centers to build,
such that population demand, policy constraints, and capacity limitations are met at minimum total cost.

## Sets and Indices
$Z:$ set of zipcodes 

$F_z:$ set of existing facilities in zip code $z$

$L_z:$ set of potential facility locations in zip code $z$

$S = \{\text{small, medium, large}\}$

| Symbol                           | Description                                               | Value                                    |
| :------------------------------- | :-------------------------------------------------------- | :----------------------------------------------- |
|      $C_f$          | Total capacity of existing facility $f \in F$               | -                                    |
| $C_f^{0-5}$       | Current capacity for ages 0–5 at facility $f \in F$         | -                                   |
| $\text{size}_s$              | Total slots for a new facility of type $s \in S$            | $\{100, 200, 400\}$                                  |
| $\text{cost}_s$              | Fixed cost to build a facility of size $s \in S$            | $\{65000, 95000, 115000\}$                                 |
| $N_z^{0-5}$                  | Population aged 0–5 in zipcode $z \in Z$                       | From census data                                 |
| $N_z^{5-12}$                 | Population aged 5–12 in zipcode $z \in Z$                       | From census data                                 |
| $N_z = N_z^{0-5} + N_z^{5-12}$ | Total children under 13                                  | —                                                |
| $E_z$                        | Employment rate in zipcode $z \in Z$                            | Employment dataset                               |
| $I_z$                       | Average household income in $                                 | Income dataset                                   |
| $\theta_z$                  | Required coverage rate (high demand = 0.5; otherwise 1/3) |$\begin{cases} 0 & x\leq 0 \\
\frac{100-x}{100} & 0\leq x\leq 100 \
\0 & 100\leq x \end{cases}$ 0.5 if \( E_z \ge 0.6 \) or \( I_z \le 60{,}000 \), else 1/3 |
| \( k \)                          | Economy-of-scale constant in per-slot cost                | 30,000                                           |
| \( c_{\min}, c_{\max} \)         | Minimum and maximum per-slot costs                        | 80, 300                                          |
| \( \bar{C} \)                    | Lower bound for capacity in cost formula                  | 100                                              |
| \( \text{limit}_f \)             | Expansion cap                                             | \( \min(0.2 \cdot \text{cap}_f, 500) \)          |
| \( \alpha \)                     | Equipment cost per 0–5 slot                               | \$100 per slot                                   |
| \( \beta_f \)                    | Fixed cost for major (≥100%) expansions                   | \$20,000 + 200·cap_f                             |


| Symbol | Description | Value  |
| :------ | :----------- | :------------- |
| $C_f$ | Total capacity of existing facility $f \in F$ | - |
| $C_f^{0-5}$ | Current capacity for ages 0–5 at facility $f \in F$ | - |
| $\text{size}_s$ | Total slots for a new facility of type $s \in S$ | $\{100, 200, 400\}$ |
| $\text{cost}_s$ | Fixed cost to build a facility of size $s \in S$ | $\{65{,}000, 95{,}000, 115{,}000\}$ |
| $N_z^{0-5}$ | Population aged 0–5 in ZIP code $z \in Z$ | - |
| $N_z^{5-12}$ | Population aged 5–12 in ZIP code $z \in Z$ | - |
| $N_z = N_z^{0-5} + N_z^{5-12}$ | Total children under 13 | — |
| $E_z$ | Employment rate in ZIP code $z \in Z$ | - |
| $I_z$ | Average household income in ZIP code $z \in Z$ | - |
| $\theta_z$ | Required coverage rate (high demand = 0.5; otherwise 1/3) | $ \theta_z = \begin{cases} 0.5, & \text{if } E_z \ge 0.6 \text{ or } I_z \le 60{,}000 \\[6pt] \frac{1}{3}, & \text{otherwise} \end{cases} $ |
| $k$ | Economy-of-scale constant in per-slot cost | $30,000$ |
| $c_{\min}, c_{\max}$ | Minimum and maximum per-slot costs | $\{80, 300\}$ |
| $\bar{C}$ | Lower bound for capacity in cost formula | $100$ |
| $\text{limit}_f$ | Expansion cap | $\min(0.2 \cdot \text{cap}_f, 500)$ |
| $\alpha$ | Equipment cost per new 0–5 slot | $ \$100$ per slot |
| $\beta_f$ | Fixed cost for major ($\geq 100%$) expansions | $\$20,000 + 200·C_f$ |


In [13]:
# --- Configuration ---
facility_sizes = {'small': 100, 'medium': 200, 'large': 400}
facility_costs = {'small': 65000, 'medium': 95000, 'large': 115000}

# --- Cost parameters for expansion ---
k = 30000           # economy-of-scale scaling factor
cap_floor = 50   # min capacity considered for pricing
slope_min = 80.0    # $/slot floor (very large sites can't be free)
slope_max = 300.0   # $/slot ceiling (tiny sites not punished too much)

# --- Helper datasets ---
zipcodes = pop['zipcode'].unique()
fac_init['facility_id'] = fac_init['facility_id'].astype(str)

results, objectives = {}, {}

In [14]:


# --- Loop over ZIPs ---
for z in zipcodes:
    m = gp.Model(f"zip_{z}")
    m.Params.OutputFlag = 0   # silence Gurobi output

    # Facilities and potential locations in this ZIP
    fz = fac_init[fac_init['zipcode'] == z]
    lz = loc[loc['zipcode'] == z]

    if fz.empty and lz.empty:
        continue  # skip ZIPs with no facility or potential site

    # --- Decision variables ---
    x = m.addVars(fz['facility_id'], lb=0, name="expand_amount")   # expansion slots
    x_05 = m.addVars(fz['facility_id'], lb=0, name="expand_under5") # 0–5 expansion slots
    c_exp = m.addVars(fz['facility_id'], lb=0, name="expansion_cost") # total expansion cost

    y = m.addVars(lz.index, facility_sizes.keys(), vtype=GRB.BINARY, name="build")  # new builds
    y_new_05 = m.addVars(lz.index, facility_sizes.keys(), lb=0, vtype=GRB.INTEGER,
                         name="build_under5")  # 0–5 slots in new builds

    # --- Expansion cost constraints (piecewise economies of scale) ---
    for f in fz.index:
        fid = fz.loc[f, 'facility_id']
        cap_f = float(fz.loc[f, 'total_cap'])
        small_limit = min(0.2 * cap_f, 500.0)  # 20% or 500 slots
        limit_f = small_limit

        # expansion feasibility
        m.addConstr(x[fid] <= limit_f, name=f"expansion_limit_{fid}")
        m.addConstr(x_05[fid] <= x[fid], name=f"under5_le_total_{fid}")

        # if no expansion possible, skip PWL and set cost = 0
        if cap_f <= 1e-6 or limit_f <= 1e-6:
            m.addConstr(c_exp[fid] == 0, name=f"no_expansion_cost_{fid}")
            continue

        # --- per-slot cost with clamping (economy of scale, bounded) ---
        eff_cap = max(cap_f, cap_floor)
        per_slot = k / eff_cap
        per_slot = max(min(per_slot, slope_max), slope_min)

        # cost if expanding ≥100%
        fixed_major = 20000 + 200 * cap_f

        # sorted breakpoints for PWL
        if limit_f >= cap_f:
            xpts = [0.0, cap_f, limit_f]
            ypts = [0.0, per_slot * cap_f, fixed_major]
        else:
            xpts = [0.0, limit_f]
            ypts = [0.0, per_slot * limit_f]

        m.addGenConstrPWL(x[fid], c_exp[fid], xpts, ypts, name=f"pwl_cost_{fid}")

    # --- New build constraints ---
    # Capacity linkage for new 0–5 slots (max 50% under age 5)
    for l in lz.index: 
        for s in facility_sizes:
            m.addConstr(
                y_new_05[l, s] <= 0.5 * facility_sizes[s] * y[l, s],
                name=f"under5_new_{l}_{s}"
            )

    # --- Demand and Policy Constraints ---
    N = pop.loc[pop['zipcode'] == z, ['pop_0_5', 'pop_5_12']].sum(axis=1).values[0]
    N_05 = pop.loc[pop['zipcode'] == z, 'pop_0_5'].values[0]
    E = emp.loc[emp['zipcode'] == z, 'employment rate'].values[0]
    I = inc.loc[inc['zipcode'] == z, 'average income'].values[0]

    high_demand = (E >= 0.6) or (I <= 60000)
    threshold = 0.5 if high_demand else 1/3

    # total available slots
    A_total = (
        quicksum(fz.loc[f, 'total_cap'] + x[fz.loc[f, 'facility_id']] for f in fz.index)
        + quicksum(facility_sizes[s] * y[l, s] for l in lz.index for s in facility_sizes)
    )
    m.addConstr(A_total >= threshold * N, name=f"demand_threshold_{z}")

    # under-5 requirement
    A_05 = (
        quicksum(fz.loc[f, 'cap_0_5'] + x_05[fz.loc[f, 'facility_id']] for f in fz.index)
        + quicksum(y_new_05[l, s] for l in lz.index for s in facility_sizes)
    )
    m.addConstr(A_05 >= (2 / 3) * N_05, name=f"under5_policy_{z}")

    # --- Objective ---
    # expansion cost + 100 per 0–5 slot
    expansion_cost = quicksum(c_exp[fz.loc[f, 'facility_id']] + 100 * x_05[fz.loc[f, 'facility_id']]
                              for f in fz.index)
    # new build cost + 100 per 0–5 slot
    build_cost = quicksum(facility_costs[s] * y[l, s] + 100 * y_new_05[l, s]
                          for l in lz.index for s in facility_sizes)

    m.setObjective(expansion_cost + build_cost, GRB.MINIMIZE)

    # --- Optimize ---
    m.optimize()
    objectives[z] = m.ObjVal if m.status == GRB.OPTIMAL else None

    # --- Save results ---
    row = {}
    if m.status in (GRB.OPTIMAL, GRB.SUBOPTIMAL):
        for v in m.getVars():
            try:
                row[v.VarName] = v.X
            except gp.GurobiError:
                row[v.VarName] = None
    else:
        row['Status'] = m.status
    results[z] = row

# === Collect & summarize results ===
res_df = pd.DataFrame(results).T
res_df['Objective'] = pd.Series(objectives)

# --- Filter feasible ZIPs ---
feasible_zips = [z for z, v in objectives.items() if v is not None]
if not feasible_zips:
    print("No feasible ZIP-level models found.")
else:
    total_cost = sum(objectives[z] for z in feasible_zips)
    new_facilities = 0
    added_slots_expansion = 0
    under5_upgraded = 0
    construction_cost = 0
    equipment_cost = 0
    expansion_cost_total = 0

    for z, row in results.items():
        for name, val in row.items():
            if val == 0 or pd.isna(val):
                continue

            # New builds
            if name.startswith("build[") and val > 0:
                new_facilities += 1
                for s in facility_sizes:
                    if f",{s}]" in name or name.endswith(f",{s}]"):
                        construction_cost += facility_costs[s] * val

            # Expansions
            elif name.startswith("expand_amount["):
                added_slots_expansion += val

            # Under-5 slots
            elif "expand_under5" in name or "build_under5" in name:
                under5_upgraded += val
                equipment_cost += 100 * val

            # PWL expansion cost variable
            elif name.startswith("expansion_cost["):
                expansion_cost_total += val

    

In [12]:
print("\n=== Optimal Solution ===")
print(f"Total cost (Minimum Budget): ${total_cost:,.2f}")
print(f"  New Facility Cost: ${construction_cost:,.2f}")
print(f"  Expansion Cost: ${expansion_cost_total:,.2f}")
print(f"  0-5 Slots Cost (0–5): ${equipment_cost:,.2f}\n")

print("Decisions:")
print(f"  Number of New Facilities: {int(new_facilities):,}")
print(f"  Additional Slots via Expansion: {int(added_slots_expansion):,}")
print(f"  Additional 0–5 Slots: {int(under5_upgraded):,}")





=== Optimal Solution ===
Total cost (Minimum Budget): $471,336,905.89
  New Facility Cost: $395,230,000.00
  Expansion Cost: $11,997,463.28
  0-5 Slots Cost (0–5): $64,109,442.61

Decisions:
  Number of New Facilities: 3,818
  Additional Slots via Expansion: 66,019
  Additional 0–5 Slots: 641,094
